Input

In [36]:
from pathlib import Path 
sample = Path('sample').read_text()
input = Path('input').read_text()

Parsing

In [37]:
import re
rx = re.compile(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)")

def parse(input):
  return [tuple(map(int, rx.match(line).groups())) for line in input.splitlines(False)]

parse(sample)

[(2, 18, -2, 15),
 (9, 16, 10, 16),
 (13, 2, 15, 3),
 (12, 14, 10, 16),
 (10, 20, 10, 16),
 (14, 17, 10, 16),
 (8, 7, 2, 10),
 (2, 0, 2, 10),
 (0, 11, 2, 10),
 (20, 14, 25, 17),
 (17, 20, 21, 22),
 (16, 7, 15, 3),
 (14, 3, 15, 3),
 (20, 1, 15, 3)]

Solution

In [38]:
def dist(x1, y1, x2, y2):
  return abs(x1 - x2) + abs(y1 - y2)

def intersect_line_and_circle(y, sx, sy, r):
  d = r - abs(y - sy)
  return None if d < 0 else (sx - d, sx + d)

def intersect_ranges(a, b):
  first = max(a[0], b[0])
  second = min(a[1] + 1, b[1] + 1)
  return (first, second) if first <= second else None

def union_ranges(a, b):
  if not intersect_ranges(a, b):
    return None
  first = min(a[0], b[0])
  second = max(a[1], b[1])
  return (first, second)

def union_sorted_ranges(ranges):
  result = [ranges[0]]
  for r in ranges[1:]:
    if intersect := union_ranges(result[-1], r):
      result[-1] = intersect
    else:
      result.append(r)
  return result

def solve(input, y):
  scans = parse(input)
  rs = [dist(sx, sy, bx, by) for sx, sy, bx, by in scans]

  ranges = [intersect_line_and_circle(y, sx, sy, r) for (sx, sy, bx, by), r in zip(scans, rs)]
  ranges = [r for r in ranges if r]
  ranges = sorted(ranges, key=lambda r: r[0])
  ranges = union_sorted_ranges(ranges)
  result = sum(r[1] - r[0] + 1 for r in ranges)

  beacons = len(set((bx, by) for sx, sy, bx, by in scans if by == y))
  result -= beacons
  
  return result

assert solve(sample, 10) == 26
solve(input, 2000000)

5335787

In [39]:
def crop_range(inner, outer):
  return (max(inner[0], outer[0]), min(inner[1], outer[1]))

def solve(input, m):
  scans = parse(input)
  rs = [dist(sx, sy, bx, by) for sx, sy, bx, by in scans]

  for y in range(m + 1):
    ranges = [intersect_line_and_circle(y, sx, sy, r) for (sx, sy, bx, by), r in zip(scans, rs)]
    ranges = [r for r in ranges if r]
    ranges = sorted(ranges, key=lambda r: r[0])
    ranges = union_sorted_ranges(ranges)
    ranges = [crop_range(r, (0, m)) for r in ranges]
    result = sum(r[1] - r[0] for r in ranges)

    if result < m:
      x = ranges[0][1] + 1
      return x * 4000000 + y

assert solve(sample, 20) == 56000011
solve(input, 4000000)

13673971349056